### Static Equilibrium

In this notebook, we will derive, interpret, and simulate the static equilibrium of the model.

Last edited: Wed. 1/23/19, by Arnav Sood

In [1]:
# For equation numbering
macro javascript_str(s) display("text/javascript", s); end
    
javascript"""
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);
"""

### Market's Problem

Begin with the formula for the Inflow Prior $\lambda^I$

\begin{equation} 
\lambda^I = \frac{e_H \lambda}{e_H \lambda + e_L (1 - \lambda}
\end{equation}

Recall the formula for the final signal-dependent prior, $\lambda^o(h; m)$, in terms of the above:

\begin{equation} 
\lambda^o(h; m) = \frac{\lambda^I}{\lambda^I + (1-\lambda^I)\phi}
\end{equation}

This can be rearranged to express $\lambda^o(h; m)$ (hereafter $\lambda^o(h)$, for convenience), in terms of model primitives:

\begin{equation} 
\lambda^o(h) = \frac{e_H \lambda}{e_H \lambda + \phi e_L (1 - \lambda)}
\end{equation}

This passes a sanity check, since it says that the final prior is the share of high types, out of all those who get the high signal (high types + $\phi$ times low types)

### Firms' Problem

Firms here are mechanistic, acting according to:

\begin{equation}
V_i = \max_{\text{entry}, \text{no entry}}\{\omega_i, \phi_i R - p\}
\end{equation}

in the constraint region.

Assuming an interior solution (that is, one where $e_H, e_L \in (0, 1)$ strictly), we have:

\begin{align}
e_H &= F_H(R - p) \\ 
e_L &= F_L(\phi R - p)
\end{align}

Note that $e_H, e_L$ are but **fractions** of firms of each type that enter the market, and need to be multiplied by their respective (prior) masses, $\lambda, 1 - \lambda$, to have meaning.

We assumed earlier that the constraint is not violated. Or:

\begin{align}
\lambda^o(h) &\geq \underline{\lambda} \\ 
\frac{e_H \lambda}{e_H \lambda + \phi e_L (1 - \lambda)} &\geq \underline{\lambda} \\ 
e_H \lambda (1 - \underline{\lambda}) &\geq \underline{\lambda} \phi e_L (1 - \lambda) \\ 
\phi &\leq \frac{e_H \lambda (1 - \underline{\lambda})}{e_L \underline{\lambda} (1 - \lambda)}
\end{align}

This will be our chief constraint. For ease of reference, we can call this the **market credulity constraint.**

### Rater's Problem

A forward-looking credit rater, who (for convenience) shares a common prior with the market about the type-distribution of firms, is then seeking to solve the following problem:


\begin{align}
\max_{\phi \in [0, 1], p \geq 0} p \left\{ F_H(R - p) \lambda + F_L(\phi R - p)(1 - \lambda) \right\}
\end{align}

subject to: 

\begin{equation}
\phi \leq \frac{e_H \lambda (1 - \underline{\lambda})}{e_L \underline{\lambda} (1 - \lambda)}
\end{equation}

We can write the following Lagrangian (assuming the problem is suitable, which we will have to show later): 

\begin{equation}
L = p \left\{ F_H(R - p) \lambda + F_L(\phi R - p)(1 - \lambda) \right\} - \mu (\log \phi + \log F_L(\phi R - p) - \log F_H(R - p) - \log \tau)
\end{equation}

Where 

\begin{equation}
\tau \equiv \frac{\lambda(1 - \underline{\lambda})}{\underline{\lambda}(1-\lambda)}
\end{equation}

We can derive first-order conditions:

#### FOC $\phi$

\begin{equation}
p R (1 - \lambda) f_L(\phi R - p) = \mu (\frac{1}{\phi} + \frac{R f_L(\phi R - p)}{F_L(\phi R - p}
\end{equation}

#### FOC $p$

\begin{equation}
F_H(R - p)\lambda + F_L(\phi R - p)(1 - \lambda) - p[f_H(R - p)\lambda + f_L(\phi R - p)(1 - \lambda)] + \mu \left(\frac{f_L(\phi R - p)}{F_L(\phi R - p)} - \frac{f_H(R - p)}{F_H(R - p)} \right) = 0
\end{equation}

#### Complementary Slackness

\begin{equation}
\mu (\log \phi + \log F_L (\phi R - p) - \log F_H (R - p) - \log \tau) = 0
\end{equation}

#### Model Solution

Rewriting (15), we can derive an expression for the Lagrange/KKT multiplier:

\begin{equation}
\mu = \frac{\phi (1 - \lambda)F_L(\phi R - p)R \cdot f_L(\phi R - p)}{F_L(\phi R - p) + \phi R f_L(\phi R - p)}
\end{equation}

We can make some observations here. 

1. The term to the left of the $\cdot$ is total payoff earned on the public market by low-type firms.

2. The multiplier is nonzero iff (1) the $\phi$ parameter is nonzero, (2) the market reward is nonzero, (3) the price is nonzero, (4) $\lambda$ is not 1 (i.e., rater and market must believe $\exists$ some low-type firms), and (5) **both** $f_L$ and $F_L$ are nonzero (i.e., there has been some entry, and the density function is not locally flat, so a small perturbation yields rewards).

Assuming the above parameter restrictions hold, we know the complementary slackness condition can be divided through by $\mu$, to yield a binding constraint.

### Interpretation

### Implementation and Comparative Statics

In general, we have a system of 3 equations (15, 16, 17) in 3 unknowns ($\phi$, $p$, $\mu$). We can feed this to a nonlinear solver.

First, define a holder for model primitives.

#### Setup and Introduction

In [2]:
using Parameters, Distributions

In [3]:
Model = @with_kw (λ = 0.5, # share of good firms
                  R = 2, # market payoff
                  λ_bar = 0.74, # market credulity threshold
                  ω_H = Uniform(1.5, 2.5), # High-type distribution of outside options
                  ω_L = Uniform(1.01, 2.01), # Low-type distribution of outside options
                  f_L = x -> pdf(ω_L, x),
                  f_H = x -> pdf(ω_H, x),
                  F_L = x -> cdf(ω_L, x),
                  F_H = x -> cdf(ω_H, x)
           )

#3 (generic function with 2 methods)

Before proceeding, it's worh examining how this object we created works.

If we call it without arguments, it will the default values we supplied.

In [4]:
Model()

(λ = 0.5, R = 2, λ_bar = 0.74, ω_H = Uniform{Float64}(a=1.5, b=2.5), ω_L = Uniform{Float64}(a=1.01, b=2.01), f_L = getfield(Main, Symbol("##5#14")){Uniform{Float64}}(Uniform{Float64}(a=1.01, b=2.01)), f_H = getfield(Main, Symbol("##6#15")){Uniform{Float64}}(Uniform{Float64}(a=1.5, b=2.5)), F_L = getfield(Main, Symbol("##7#16")){Uniform{Float64}}(Uniform{Float64}(a=1.01, b=2.01)), F_H = getfield(Main, Symbol("##8#17")){Uniform{Float64}}(Uniform{Float64}(a=1.5, b=2.5)))

But we can also supply specific arguments.

In [5]:
Model(λ = 0.3, R = 20, ω_H = Uniform(1., 21.))

(λ = 0.3, R = 20, λ_bar = 0.74, ω_H = Uniform{Float64}(a=1.0, b=21.0), ω_L = Uniform{Float64}(a=1.01, b=2.01), f_L = getfield(Main, Symbol("##9#18")){Uniform{Float64}}(Uniform{Float64}(a=1.01, b=2.01)), f_H = getfield(Main, Symbol("##10#19")){Uniform{Float64}}(Uniform{Float64}(a=1.0, b=21.0)), F_L = getfield(Main, Symbol("##11#20")){Uniform{Float64}}(Uniform{Float64}(a=1.01, b=2.01)), F_H = getfield(Main, Symbol("##12#21")){Uniform{Float64}}(Uniform{Float64}(a=1.0, b=21.0)))

It is preferred to use these things instead of a struct (i.e., the code below) for two reasons. 

1. Simplicity. Compare the code above to:

```
struct Model{TF <: AbstractFloat, TI <: Integer, ...}
λ::TF 
R::TI
...
```

2. Correctness. If you are writing a `struct` by hand, then to get equal performance to a named tuple (the kind of thing we generate with a `Model()` call), we need to get the type parameterization (i.e., the `TF, TI`, etc.) exactly right. This is complicated by the fact that we use objects that are themselves parametrically typed (such as `Uniform{Float64}(a = 1.01, b = 2.01)`, for the distributions).

#### Computational Solution

Define a baseline model.

In [69]:
m = Model(R = 1.5);

Write code to take guesses for $\phi, p, \mu$, and a model $m$, and calculate the residuals for the three equations.

In [70]:
function residuals(vals; model = m)
    # unpack inputs
    @unpack λ, R, λ_bar, ω_H, ω_L, f_L, f_H, F_L, F_H = model
    ϕ, p, μ = vals # so vals is some vector [ϕ, p, μ]
    # define τ
    τ = λ*(1 - λ_bar)/(λ_bar * (1 - λ))
    # calculate residuals
    ϕ_residual = ϕ > 0 && F_L(ϕ*R - p) > 0 ? p*R*(1-λ)*f_L(ϕ*R - p) - μ*(1/ϕ + R*f_L(ϕ*R - p)/F_L(ϕ*R -p)) : 100 # (15) (return 100 for invalid residual expressions)
    p_residual = F_L(ϕ*R - p) > 0 && F_H(R - p) > 0 ? F_H(R - p)*λ + F_L(ϕ*R - p)*(1 - λ) - p*(f_H(R - p) + f_L(ϕ*R - p)) + μ*(f_L(ϕ*R - p)/F_L(ϕ*R - p) - f_H(R - p)/F_H(R - p)) : 100 # (16)
    μ_residual = ϕ > 0 && \tau > 0 && F_L(ϕ*R - p) > 0 && F_H(R - p) > 0 ? μ*(log(ϕ) + log(F_L(ϕ*R - p)) - log(F_H(R - p)) - log(τ)) : 100 # (17)
    # return 
    residuals = [ϕ_residual, p_residual, μ_residual]
end

residuals (generic function with 1 method)

Pass this to the solver.

In [71]:
using NLsolve # https://github.com/JuliaNLSolvers/NLsolve.jl, one of the main Julia solvers

In [72]:
@time result = nlsolve(residuals, [0.001, 0.001, 0.], inplace = false, store_trace = true, iterations = 10^6)

  1.850637 seconds (10.21 M allocations: 1.062 GiB, 40.34% gc time)


Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.001, 0.001, 0.0]
 * Zero: [NaN, NaN, NaN]
 * Inf-norm of residuals: 100.000000
 * Iterations: 1000000
 * Convergence: false
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: false
 * Function Calls (f): 1000001
 * Jacobian Calls (df/dx): 1